# Lab 2 & 3

## Importing and using ResNet predefined model

In [2]:
from torchvision.models import resnet18
from torch import nn
import torch
import torch.nn.functional as F
import torch.optim as optim

In [3]:
model=resnet18(num_classes=10)
# model=resnet18(num_classes=10,pretrained=True) # Import with weights
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
model.conv1=nn.Conv2d(1,64,kernel_size=(7,7),stride=(2,2),padding=(3,3),bias=False)

In [5]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

img_transform=transforms.Compose([transforms.ToTensor()
                                  ])

train_dataset=MNIST(root="./data/MNIST",download=True,train=True,transform=img_transform)
train_dataloader=DataLoader(train_dataset,batch_size=128,shuffle=True)

test_dataset=MNIST(root="./data/MNIST",download=True,train=False,transform=img_transform)
test_dataloader=DataLoader(test_dataset,batch_size=128,shuffle=True)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



In [7]:
device="cuda:0"

In [48]:
epochs=10
train_loss_avg=[]
optimizer=optim.Adam(model.parameters(),lr=1e-3)
lossfn=nn.CrossEntropyLoss()
model.to(device)
for e in range(epochs):
  num_batches=0
  for image,label in train_dataloader:
    image=image.to(device)
    label=label.to(device)
    pred_label=model(image)

    loss=lossfn(pred_label,label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss_avg.append(loss.item())
    num_batches+=1
  print("The loss at epoch %d is: %f"%(e+1,train_loss_avg[-1]))

The loss at epoch 1 is: 0.058839
The loss at epoch 2 is: 0.153435
The loss at epoch 3 is: 0.043546
The loss at epoch 4 is: 0.091697
The loss at epoch 5 is: 0.024640
The loss at epoch 6 is: 0.024444
The loss at epoch 7 is: 0.076988
The loss at epoch 8 is: 0.057127
The loss at epoch 9 is: 0.006009
The loss at epoch 10 is: 0.058432


# Using multiple workers for data loader and pinning the data into GPU

In [13]:
import time
train_dataset=MNIST(root="./data/MNIST/",download=True,train=True,transform=img_transform)
train_dataloader=DataLoader(train_dataset,batch_size=128,shuffle=True)

start=time.time()
for _,_ in train_dataloader:
  _=_.to(device)

print(time.time()-start)

7.13577675819397


In [20]:
import time
train_dataset=MNIST(root="./data/MNIST/",download=True,train=True,transform=img_transform)
train_dataloader=DataLoader(train_dataset,batch_size=128,shuffle=True,num_workers=2,pin_memory=False)

start=time.time()
for _,_ in train_dataloader:
  _=_.to(device)

print(time.time()-start)

6.757227659225464


In [21]:
import time
train_dataset=MNIST(root="./data/MNIST/",download=True,train=True,transform=img_transform)
train_dataloader=DataLoader(train_dataset,batch_size=128,shuffle=True,num_workers=2,pin_memory=True)

start=time.time()
for _,_ in train_dataloader:
  #_=_.to(device)
  pass

print(time.time()-start)

6.803935766220093


# Efficient way of setting gradient to zero

In [22]:
model=resnet18(num_classes=10).to(device)

In [23]:
start=time.time()
model.zero_grad()
print(time.time()-start)

0.00106048583984375


In [24]:
start=time.time()
for param in model.parameters():
  param.grad=None
print(time.time()-start)

0.0004734992980957031


In [ ]:
with torch.no_grad():
  output=model(input)

In [ ]:
@torch.no_grad()
def validation(model,input):
  out=model(input)
  return out

# Fuse pointwise operations

In [ ]:
@torch.jit.script
def fused_gelu(x):
  return x*0.5*(1.0 + torch.erf(x/1.4))

# GPU specific optimization

In [26]:
import torch
import torch.nn as nn
import time 

start=time.time()
x=torch.rand((128,3,200,300))
x=x.to(device)
print(time.time()-start)

0.21984148025512695


In [27]:
start=time.time()
x=torch.rand((128,3,200,300),device=device)
print(time.time()-start)

0.006306171417236328


In [29]:
x=torch.rand((128,3,200,200),device=device)
conv=nn.Conv2d(3,64,3,device=device)

start=time.time()
x=conv(x)
print(time.time()-start)

0.010257482528686523


In [30]:
torch.backends.cudnn.benchmark=True
x=torch.rand((128,3,200,200),device=device)
conv=nn.Conv2d(3,64,3,device=device)

start=time.time()
x=conv(x)
print(time.time()-start)

0.004571437835693359


# Mixed precision for faster training

In [45]:
from torch import nn
from torchvision.models import resnet18
from torch.cuda.amp import autocast,GradScaler
model=resnet18(num_classes=10)

In [47]:
optimizer=optim.SGD(model.parameters(),lr=1e-3)
scaler=GradScaler()
loss_fn=nn.CrossEntropyLoss()
epochs=10
model.conv1=nn.Conv2d(1,64,kernel_size=(7,7),stride=(2,2),padding=(3,3),bias=False)

model=model.to(device)

for epoch in range(epochs):
  for image,label in train_dataloader:
    image,label=image.to(device),label.to(device)
    for param in model.parameters():
      param.grad=None
    with autocast():
      output=model(image)
      loss=loss_fn(output,label)
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update() 
  print("The loss at epoch %d is: %f"%(epoch+1,loss.item()))

The loss at epoch 1 is: 0.353883
The loss at epoch 2 is: 0.282745
The loss at epoch 3 is: 0.222130
The loss at epoch 4 is: 0.211314
The loss at epoch 5 is: 0.106056
The loss at epoch 6 is: 0.036230
The loss at epoch 7 is: 0.114110
The loss at epoch 8 is: 0.132812
The loss at epoch 9 is: 0.056218
The loss at epoch 10 is: 0.061056
